In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bethel,US,2022-06-29 20:25:56,41.3712,-73.4140,85.73,45,20,10.36
1,1,Ayan,RU,2022-06-29 20:25:56,56.4500,138.1667,51.69,94,67,6.08
2,2,Saint-Pierre,RE,2022-06-29 20:25:57,-21.3393,55.4781,65.88,72,75,8.05
3,3,Puri,IN,2022-06-29 20:25:57,19.8000,85.8500,82.92,81,100,19.13
4,4,Husavik,IS,2022-06-29 20:25:58,66.0449,-17.3389,51.46,85,90,4.43


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds
                                , dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bethel,US,2022-06-29 20:25:56,41.3712,-73.4140,85.73,45,20,10.36
3,3,Puri,IN,2022-06-29 20:25:57,19.8000,85.8500,82.92,81,100,19.13
11,11,Aksu,CN,2022-06-29 20:26:01,41.1231,80.2644,82.22,26,1,6.82
12,12,Cayenne,GF,2022-06-29 20:26:02,4.9333,-52.3333,75.24,1,40,4.61
13,13,Kihei,US,2022-06-29 20:25:19,20.7850,-156.4656,82.42,69,20,21.85
19,19,Cockburn Town,TC,2022-06-29 20:26:08,21.4612,-71.1419,82.71,75,15,19.93
23,23,Hithadhoo,MV,2022-06-29 20:26:10,-0.6000,73.0833,83.32,75,75,11.03
25,25,Sulangan,PH,2022-06-29 20:26:12,11.1373,123.7228,83.25,75,100,9.44
26,26,Jamestown,US,2022-06-29 20:26:13,42.0970,-79.2353,78.53,55,40,9.22
29,29,Dromolaxia,CY,2022-06-29 20:26:15,34.8761,33.5860,81.99,50,0,11.50


In [13]:
preferred_cities_df.count()

City_ID       202
City          202
Country       202
Date          202
Lat           202
Lng           202
Max Temp      202
Humidity      202
Cloudiness    202
Wind Speed    202
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Bethel,US,85.73,41.3712,-73.4140,
3,Puri,IN,82.92,19.8000,85.8500,
11,Aksu,CN,82.22,41.1231,80.2644,
12,Cayenne,GF,75.24,4.9333,-52.3333,
13,Kihei,US,82.42,20.7850,-156.4656,
19,Cockburn Town,TC,82.71,21.4612,-71.1419,
23,Hithadhoo,MV,83.32,-0.6000,73.0833,
25,Sulangan,PH,83.25,11.1373,123.7228,
26,Jamestown,US,78.53,42.0970,-79.2353,
29,Dromolaxia,CY,81.99,34.8761,33.5860,


In [21]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Bethel,US,85.73,41.3712,-73.4140,Courtyard by Marriott Danbury
3,Puri,IN,82.92,19.8000,85.8500,OYO 1577 Hotel Shree Hari Grand
11,Aksu,CN,82.22,41.1231,80.2644,Pudong Holiday Hotel
12,Cayenne,GF,75.24,4.9333,-52.3333,Hôtel Le Dronmi
13,Kihei,US,82.42,20.7850,-156.4656,Maui Sunset
...,...,...,...,...,...,...
566,Daru,PG,78.37,-9.0763,143.2092,Daru Lodge
567,Palu,ID,75.94,-0.8917,119.8707,Hotel Santika Palu
569,Una,IN,83.41,20.8167,71.0333,Hotel Harbhole
571,Weiser,US,85.32,44.2510,-116.9693,The Colonial Motel


In [26]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))